In [1]:
import numpy as np
import sys
import glob 
import os 
from pathlib import Path

sys.path.insert(0, 'C:\\Suite2P\\')
import suite2p

from suite2p.run_s2p import run_s2p

mouseName = 'GF199'

fast_diskpath = str(Path('D:\Foustoukos'))
save_path = str(Path('D:\Foustoukos' + '\\' +  mouseName))

print(type(fast_diskpath))

ops = {
        # file paths
        'look_one_level_down': False, # whether to look in all subfolders when searching for tiffs
        'fast_disk': fast_diskpath, # used to store temporary binary file, defaults to save_path0
        'delete_bin': False, # whether to delete binary file after processing
        'mesoscan': False, # for reading in scanimage mesoscope files
        'h5py': [], # take h5py as input (deactivates data_path)
        'h5py_key': 'data', #key in h5py where data array is stored
        'save_path0': save_path, # stores results, defaults to first item in data_path
        'subfolders': [],
        'diameter' : 15,
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'tau':  0.7, # this is the main parameter for deconvolution
        'fs': 30.,  # sampling rate (PER PLANE - e.g. if you have 12 planes then this should be around 2.5)
        'force_sktiff': False, # whether or not to use scikit-image for tiff reading
        # output settings
        'preclassify': 0.5, # apply classifier before signal extraction with probability 0.5 (turn off with value 0)
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        'aspect': 1.0, # um/pixels in X / um/pixels in Y (for correct aspect ratio in GUI)
        # bidirectional phase offset
        'do_bidiphase': False,
        'bidiphase': 0,
        # registration settings
        'do_registration': 1, # whether to register data (2 forces re-registration)
        'keep_movie_raw': False,
        'nimg_init': 300, # subsampled frames for finding reference image
        'batch_size': 500, # number of frames per batch
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': False, # whether to save registered tiffs
        'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
        'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
        'pad_fft': False,
        # non rigid registration settings
        'nonrigid': True, # whether to use nonrigid registration
        'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
        'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
        'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid
        # 1P settings
        '1Preg': False, # whether to perform high-pass filtering and tapering
        'spatial_hp': 50, # window for spatial high-pass filtering before registration
        'pre_smooth': 2, # whether to smooth before high-pass filtering before registration
        'spatial_taper': 50, # how much to ignore on edges (important for vignetted windows, for FFT padding do not set BELOW 3*ops['smooth_sigma'])
        # cell detection settings
        'roidetect': True, # whether or not to run ROI extraction
        'spatial_scale': 0, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'nbinned': 5000, # max number of binned frames for cell detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'threshold_scaling': 5., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)
        # ROI extraction parameters
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)
        # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
        'chan2_thres': 0.65, # minimum for detection of brightness on channel 2
        # deconvolution settings
        'baseline': 'maximin', # baselining mode (can also choose 'prctile')
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
        'xrange': np.array([0, 0]),
        'yrange': np.array([0, 0]),
      }




<class 'str'>


In [ ]:
# provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)

datapath = Path('\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\Imaging\\')
datapath = str(datapath)
imaging_days = glob.glob(os.path.join(datapath, "GF*"))
binpath = Path('D:\Foustoukos\BIN')
binpath = str(binpath)

for n,f in enumerate(imaging_days):
    imaging_days[n] = os.path.basename(f)
    
    
for n in range(len(imaging_days)):
    
    current_day_path = str(datapath) + '\\' + str(imaging_days[n]) +'\\'
    
    binpath = Path('D:\Foustoukos\BIN' + '\\' + mouseName + '\\' + str(imaging_days[n]))
    
    binpath = str(binpath)
    
    savepath = Path('D:\Foustoukos' + '\\' +  mouseName + '\\' + str(imaging_days[n]))
    
    savepath = str(savepath)
   
    print(current_day_path)
    
    db = {
          'h5py': [], # a single h5 file path
          'h5py_key': 'data',
          'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
          'data_path': [current_day_path], # a list of folders with tiffs 
                                                 # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)

          'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
          'fast_disk': binpath, # string which specifies where the binary file will be stored (should be an SSD)
          'save_path0' : savepath,
        }


    #print(db['data_path'])
    
    opsEnd=run_s2p(ops=ops,db=db)




\\sv1files.epfl.ch\Petersen-Lab\data\GF199\Recordings\Imaging\GF199_02082019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF199\\Recordings\\Imaging\\GF199_02082019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF199\\GF199_02082019', 'save_path0': 'D:\\Foustoukos\\GF199\\GF199_02082019'}
** Found 389 tifs - converting to binary **
time 1144.59 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 13.95 sec.
2500/67497 frames, 78.79 sec.
5000/67497 frames, 159.39 sec.
7500/67497 frames, 237.29 sec.
10000/67497 frames, 318.01 sec.
12500/67497 frames, 397.91 sec.
15000/67497 frames, 479.58 sec.
17500/67497 frames, 560.94 sec.
20000/67497 frames, 642.56 sec.
22500/67497 frames, 722.27 sec.
25000/67497 frames, 800.56 sec.
27500/67497 frames, 876.61 sec.
30000/67497 frames, 953.59 sec.
32500/67497 frames, 1030.13 sec.
35000

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 6.24 sec.
Extracted fluorescence from 426 ROIs in 67497 frames, 138.55 sec.
----------- Total 290.29 sec.
----------- SPIKE DECONVOLUTION
----------- Total 56.98 sec.
Plane 0 processed in 2494.88 sec (can open in GUI).
total = 3639.47 sec.
TOTAL RUNTIME 3640.08 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF199\Recordings\Imaging\GF199_03082019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF199\\Recordings\\Imaging\\GF199_03082019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF199\\GF199_03082019', 'save_path0': 'D:\\Foustoukos\\GF199\\GF199_03082019'}
** Found 547 tifs - converting to binary **
time 1715.13 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 14.03 sec.
2500/90131 frames, 77.62 sec.
5000/90131 frames, 159.25 sec.
7500/90131 frames, 239.41 sec.
10000/90131 frames, 316.69 sec.
12500/90131 fr

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 5.66 sec.
Extracted fluorescence from 397 ROIs in 90131 frames, 195.30 sec.
----------- Total 425.07 sec.
----------- SPIKE DECONVOLUTION
----------- Total 67.56 sec.
Plane 0 processed in 3337.98 sec (can open in GUI).
total = 5053.11 sec.
TOTAL RUNTIME 5053.78 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF199\Recordings\Imaging\GF199_04082019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF199\\Recordings\\Imaging\\GF199_04082019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF199\\GF199_04082019', 'save_path0': 'D:\\Foustoukos\\GF199\\GF199_04082019'}
** Found 589 tifs - converting to binary **
time 1767.69 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 14.16 sec.
2500/98197 frames, 80.26 sec.
5000/98197 frames, 167.02 sec.
7500/98197 frames, 246.01 sec.
10000/98197 frames, 325.33 sec.
12500/98197 fr